In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Project

'Copy of DIAGNOSES_ICD.csv'   DIAGNOSES_ICD.csv.gz   sand_144.py
 DIAGNOSES_ICD.csv	      __pycache__


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Project')
from sand_144 import SAnD

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ADMISSIONS.csv to ADMISSIONS.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving PATIENTS.csv to PATIENTS.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DIAGNOSES_ICD.csv to DIAGNOSES_ICD.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ICUSTAYS.csv to ICUSTAYS.csv


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
import numpy as np

# **Calculate the Hospital stay length**

In [ ]:
admissions_csv_path = "ADMISSIONS.csv"
patients_csv_path = "PATIENTS.csv"
diags_csv_path = "DIAGNOSES_ICD.csv"
icu_csv_path = "ICUSTAYS.csv"

admissions_df = pd.read_csv(admissions_csv_path)
patients_df = pd.read_csv(patients_csv_path)
diags_df = pd.read_csv(diags_csv_path)
icu_df = pd.read_csv(icu_csv_path)

admissions_df['DISCHTIME'].fillna('0',inplace=True)
icu_df.OUTTIME.fillna('0',inplace=True)

# Convert admission and discharge times to datetime objects
admissions_df['ADMITTIME'] = pd.to_datetime(admissions_df['ADMITTIME'])
admissions_df['DISCHTIME'] = pd.to_datetime(admissions_df['DISCHTIME'])

# Calculate the length of stay for each patient
admissions_df['LENGTH_OF_STAY'] = admissions_df['DISCHTIME'] - admissions_df['ADMITTIME']

# Display the updated dataframe
print(admissions_df.head())
print ( len(admissions_df))

# Check for null values in the 'LENGTH_OF_STAY' column
null_length_of_stay = admissions_df['LENGTH_OF_STAY'].isnull()

# Print the result
print("Are there any null values in the 'LENGTH_OF_STAY' column?", null_length_of_stay.any())

   ROW_ID  SUBJECT_ID  HADM_ID           ADMITTIME           DISCHTIME  \
0      21          22   165315 2196-04-09 12:26:00 2196-04-10 15:54:00   
1      22          23   152223 2153-09-03 07:15:00 2153-09-08 19:10:00   
2      23          23   124321 2157-10-18 19:34:00 2157-10-25 14:00:00   
3      24          24   161859 2139-06-06 16:14:00 2139-06-09 12:48:00   
4      25          25   129635 2160-11-02 02:06:00 2160-11-05 14:55:00   

  DEATHTIME ADMISSION_TYPE         ADMISSION_LOCATION  \
0       NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
1       NaN       ELECTIVE  PHYS REFERRAL/NORMAL DELI   
2       NaN      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
3       NaN      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
4       NaN      EMERGENCY       EMERGENCY ROOM ADMIT   

          DISCHARGE_LOCATION INSURANCE LANGUAGE           RELIGION  \
0  DISC-TRAN CANCER/CHLDRN H   Private      NaN       UNOBTAINABLE   
1           HOME HEALTH CARE  Medicare      NaN           CATHOLIC   
2 

In [ ]:
# First, convert the TimeDelta column to total seconds, and then to days
admissions_df['LENGTH_OF_STAY_DAYS'] = admissions_df['LENGTH_OF_STAY'].dt.total_seconds() / (24 * 60 * 60)

# Convert the 'LENGTH_OF_STAY_DAYS' column to float
admissions_df['LENGTH_OF_STAY_DAYS'] = admissions_df['LENGTH_OF_STAY_DAYS'].astype(float)


# Define a function to create the buckets
def create_bucket(length_of_stay):
    if length_of_stay < 1:
        return 1
    elif 1 <= length_of_stay <= 7:
        return int(length_of_stay) + 1
    elif 7 < length_of_stay <= 14:
        return 9
    else:
        return 10

# Apply the function to the 'LENGTH_OF_STAY_DAYS' column to create a new 'Length_of_stay_Bucket' column
admissions_df['Length_of_stay_Bucket'] = admissions_df['LENGTH_OF_STAY_DAYS'].apply(create_bucket)
# Display the updated dataframe
print(admissions_df.head())
print ( len(admissions_df))

#admissions_df = admissions_df.loc[~admissions_df['Length_of_stay_Bucket'].isin([5,6,7,8,9, 10])]
#admissions_df = admissions_df.loc[~admissions_df['Length_of_stay_Bucket'].isin([10])]
#print(admissions_df.head())
#print ( len(admissions_df))


   ROW_ID  SUBJECT_ID  HADM_ID           ADMITTIME           DISCHTIME  \
0      21          22   165315 2196-04-09 12:26:00 2196-04-10 15:54:00   
1      22          23   152223 2153-09-03 07:15:00 2153-09-08 19:10:00   
2      23          23   124321 2157-10-18 19:34:00 2157-10-25 14:00:00   
3      24          24   161859 2139-06-06 16:14:00 2139-06-09 12:48:00   
4      25          25   129635 2160-11-02 02:06:00 2160-11-05 14:55:00   

  DEATHTIME ADMISSION_TYPE         ADMISSION_LOCATION  \
0       NaN      EMERGENCY       EMERGENCY ROOM ADMIT   
1       NaN       ELECTIVE  PHYS REFERRAL/NORMAL DELI   
2       NaN      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
3       NaN      EMERGENCY  TRANSFER FROM HOSP/EXTRAM   
4       NaN      EMERGENCY       EMERGENCY ROOM ADMIT   

          DISCHARGE_LOCATION INSURANCE  ... MARITAL_STATUS ETHNICITY  \
0  DISC-TRAN CANCER/CHLDRN H   Private  ...        MARRIED     WHITE   
1           HOME HEALTH CARE  Medicare  ...        MARRIED     WHITE

In [ ]:
print(patients_df.head())
print ( len(patients_df))

   ROW_ID  SUBJECT_ID GENDER                  DOB                  DOD  \
0     234         249      F  2075-03-13 00:00:00                  NaN   
1     235         250      F  2164-12-27 00:00:00  2188-11-22 00:00:00   
2     236         251      M  2090-03-15 00:00:00                  NaN   
3     237         252      M  2078-03-06 00:00:00                  NaN   
4     238         253      F  2089-11-26 00:00:00                  NaN   

              DOD_HOSP DOD_SSN  EXPIRE_FLAG  
0                  NaN     NaN            0  
1  2188-11-22 00:00:00     NaN            1  
2                  NaN     NaN            0  
3                  NaN     NaN            0  
4                  NaN     NaN            0  
46520


In [ ]:
print(diags_df.head())
print ( len(diags_df))

   ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
0    1297         109   172335      1.0     40301
1    1298         109   172335      2.0       486
2    1299         109   172335      3.0     58281
3    1300         109   172335      4.0      5855
4    1301         109   172335      5.0      4254
651047


In [ ]:
# Merge the dataframes on the 'SUBJECT_ID' column
#merged_df = pd.merge(admissions_df, patients_df, diags_df, on='SUBJECT_ID')
merged_df_1 = pd.merge(admissions_df, patients_df, on='SUBJECT_ID')
merged_df = pd.merge(merged_df_1, diags_df, on='SUBJECT_ID')

# Select the desired columns
selected_columns = ['SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'Length_of_stay_Bucket', 'ICD9_CODE']
new_dataset = merged_df[selected_columns]
#new_dataset = merged_df[selected_columns].copy()

# Display the new dataset
print(new_dataset.head())
print ( len(new_dataset))

# Create the all_diags list
all_diags = new_dataset['ICD9_CODE'].tolist()

print(len(all_diags))

# Create a list of length of stay in days (as floats)
los = new_dataset['Length_of_stay_Bucket'].tolist()

# Get the unique values
unique_values = set(los)

# Print the unique values
print("Unique values in 'los':", unique_values)
print(len(los))

   SUBJECT_ID GENDER                  DOB  DOD  Length_of_stay_Bucket  \
0          22      F  2131-05-07 00:00:00  NaN                      2   
1          22      F  2131-05-07 00:00:00  NaN                      2   
2          22      F  2131-05-07 00:00:00  NaN                      2   
3          22      F  2131-05-07 00:00:00  NaN                      2   
4          22      F  2131-05-07 00:00:00  NaN                      2   

  ICD9_CODE  
0      9678  
1      9693  
2     E9502  
3     E9503  
4      3488  
1374979
1374979
Unique values in 'los': {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
1374979


In [ ]:
print(len(new_dataset))

1374979


# **Create custom Dataset to get sequences and target values (length_of_stay)**

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# Preprocess your data (You should preprocess the data to create sequences and targets)

# Example: sequences, targets = preprocess(new_dataset)

class CustomDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
        
    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        sequence = self.sequences[index]
        target = self.targets[index]

        #return torch.tensor(sequence, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)
        return sequence,target

unique_labels1 = set(los)
print(f"Unique class labels: {unique_labels1}")

los_zero_indexed = [x - 1 for x in los]
dataset = CustomDataset(all_diags, los_zero_indexed)
#dataset = CustomDataset(all_diags, los)
unique_labels = set(los_zero_indexed)
print(f"Unique class labels: {unique_labels}")


Unique class labels: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Unique class labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


# **Create Train , Validation and Test dataset**

In [ ]:
from torch.utils.data.dataset import random_split

split = int(len(dataset)*0.7)

lengths = [split, len(dataset) - split]
train_dataset, val_test_dataset = random_split(dataset, lengths)


split = int(len(val_test_dataset)*0.7)
lengths = [split, len(val_test_dataset) - split]
val_dataset, test_dataset = random_split(val_test_dataset, lengths)


print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))
print("Length of test dataset:", len(test_dataset))

Length of train dataset: 4
Length of val dataset: 2
Length of test dataset: 1


In [ ]:
#Get dictionary index for all ICD9 codes
icd_uniq = diags_df.ICD9_CODE.unique()
# icd_uniq = ['A', 'B', 'A']
icd_dic = {}
i = 0
for code in icd_uniq:
    icd_dic[code] = i
    i += 1
len(icd_dic)

6985

In [ ]:
def collate_fn(data):
    
    #sequences, labels = zip(*data)
    sequences, length_of_stay_Bucket = zip(*data)
    
    # Create tensors for length_of_stay
    y = torch.tensor(length_of_stay_Bucket, dtype=torch.int64)

    #num_patients = len(sequences)
    #num_visits = [len(patient) for patient in sequences]
    #num_codes = [len(visit) for patient in sequences for visit in patient]


    def is_iterable(obj):
         try:
             iter(obj)
             return True
         except TypeError:
              return False

    num_patients = len(sequences)
    #num_visits = [len(patient) for patient in sequences if is_iterable(patient)]
    #num_codes = [len(visit) for patient in sequences for visit in patient]
    #num_codes = [len(visit) for patient in sequences for visit in patient if is_iterable(patient) and is_iterable(visit)]


    max_num_visits = 42 #max(num_visits)
    max_num_codes = 39 #max(num_codes)
#     print(max_num_visits, max_num_codes, len(y) )
    #x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.float)
    x = torch.zeros((num_patients, max_num_codes), dtype=torch.float)
    for i_patient, patient in enumerate(sequences):
        #for j_visit, visit in enumerate(patient):
            for k_diag, diag in enumerate(sequences):
                if diag != diag:
#                     #print(diag, i_patient, j_visit,k_diag) 
                    continue
                #else: x[i_patient, j_visit,k_diag ] =icd_dic[diag]
                else: x[i_patient, k_diag] = icd_dic.get(diag, 0)
    #x = x.type(torch.LongTensor)
    #y = y.type(torch.LongTensor)
    return x, y
    

# **Load the inital data into train_loader, val_loader, test_loader**

In [ ]:
def load_data(train_dataset, val_dataset, test_dataset, batch_size, collate_fn):
    batch_size = batch_size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)
    
    return train_loader, val_loader, test_loader

batch_size = 128
train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset, batch_size, collate_fn)

in_feature = 39
seq_len = 42
n_heads = 32
factor = 5 # M - dense interpolation factor
num_class = 10
num_layers = 1 #N

In [ ]:
from sand_144 import SAnD, MyNeuralNetworkClassifier

# **Import Sand and relevant classes to run our Training and evaluate job**

In [ ]:

clf = MyNeuralNetworkClassifier(
    SAnD(in_feature, seq_len, n_heads, factor, num_class, num_layers),
    nn.CrossEntropyLoss(),
    optim.Adam, optimizer_config={"lr": 1e-5, "betas": (0.9, 0.98), "eps": 4e-09, "weight_decay": 5e-4},
)

In [ ]:
# training network
clf.fit(
    {"train": train_loader,
     "val": val_loader},
     epochs= 1
)



  0%|          | 0/4 [00:00<?, ?it/s]

TypeError: ignored